# Make a catalog of SEDs

In [113]:
# Imports
from SEDkit import sed, spectrum, synphot
import astropy.units as q
import numpy as np
from pkg_resources import resource_filename
from bokeh.io import output_notebook, show
import copy
output_notebook()

Loading BokehJS ...

First we initialize an empty catalog.

In [126]:
# Make an SED catalog
cat = sed.SEDCatalog()

Now let's make a few SEDs

In [127]:
# Make an SED
s1 = sed.SED(parallax=(1.7474*q.mas, 0.0501*q.mas), spectral_type='M0V', name='2MASS 261832886')
s1.add_photometry_file(resource_filename('SEDkit', 'data/M0V_photometry.txt'))
s1.make_sed()
cat.add_SED(s1)

Make this handle asymmetric uncertainties!
Setting age to (<Quantity 6.0 Gyr>, <Quantity 4.0 Gyr>)
Setting radius to (<Quantity 0.8786209573091851 solRad>, <Quantity 0.06782337214316517 solRad>)


In [128]:
# Make 30 other SEDs (I'll just change the parallax)
for i in range(10):
    s2 = copy.copy(s1)
    s2.parallax = abs(np.random.normal(loc=1.7474, scale=0.1))*q.mas, 0.1*q.mas
    s2.make_sed()
    cat.add_SED(s2)

Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!


Then the aggregated results look like this:

In [129]:
cat.results

name,age,age_unc,distance,distance_unc,parallax,parallax_unc,radius,radius_unc,spectral_type,spectral_type_unc,membership,fbol,fbol_unc,mbol,mbol_unc,Lbol,Lbol_unc,Lbol_sun,Lbol_sun_unc,Mbol,Mbol_unc,logg,logg_unc,mass,mass_unc,Teff,Teff_unc
,Myr,Myr,pc,pc,mas,mas,6.957e+08 m,,,,,erg / (cm2 s),erg / (cm2 s),,,erg / s,erg / s,,,,,,,solMass,solMass,K,K
object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
2MASS 261832886,6000.0,4000.0,572.28,16.41,1.7474,0.0501,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.66179055973561e+32,2.1287295997847215e+31,-1.019,0.025,7.304,0.033,4.5,0.07,0.8896913720017506,0.0,3424.0,141.0
2MASS 261832886,6000.0,4000.0,546.14,29.83,1.8310332452038867,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.33491167188553e+32,3.65683145220767e+31,-1.06,0.048,7.405,0.06,4.5,0.07,0.8896913720017506,0.0,3345.0,158.0
2MASS 261832886,6000.0,4000.0,557.76,31.11,1.7928914424681888,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.478332499784636e+32,3.8942868435410006e+31,-1.042,0.049,7.36,0.061,4.5,0.07,0.8896913720017506,0.0,3380.0,161.0
2MASS 261832886,6000.0,4000.0,532.68,28.38,1.8772833701522362,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.1725548981095634e+32,3.3939846875197012e+31,-1.082,0.046,7.46,0.059,4.5,0.07,0.8896913720017506,0.0,3303.0,155.0
2MASS 261832886,6000.0,4000.0,571.38,32.65,1.7501578971253309,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.650282137037149e+32,4.186149083474423e+31,-1.021,0.05,7.307,0.063,4.5,0.07,0.8896913720017506,0.0,3421.0,164.0
2MASS 261832886,6000.0,4000.0,548.19,30.05,1.824195995718835,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.35999461712158e+32,3.697528699709914e+31,-1.057,0.048,7.397,0.06,4.5,0.07,0.8896913720017506,0.0,3351.0,158.0
2MASS 261832886,6000.0,4000.0,546.8,29.9,1.8288234219701291,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.342976899345008e+32,3.669810907809484e+31,-1.059,0.048,7.403,0.06,4.5,0.07,0.8896913720017506,0.0,3347.0,158.0
2MASS 261832886,6000.0,4000.0,609.42,37.14,1.6408938060403373,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,4.152501226694657e+32,5.076733230446691e+31,-0.965,0.053,7.167,0.067,4.5,0.07,0.8896913720017506,0.0,3533.0,173.0


We can also plot the results by column name.

In [133]:
show(cat.plot('distance', 'Teff'))